In [1]:
import sys, os, json
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
import itertools
from gensim.corpora import Dictionary
from collections import defaultdict
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow, experiment_result
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper


In [2]:
def combine(params):
    param_names = list(params.keys())
    param_values = list(params.values())
    param_combinations = list(itertools.product(*param_values))
    return param_combinations

In [3]:
lda_param={'num_topics': range(5,20), 'decay': [0.5,0.75,0.9], 'passes': [1,2,5]}

hdp_param={'kappa': [1,1.05,1.1], 'K': [10, 15, 30], 'T': [100,150,200],
           'alpha': [0.9,1,1.1], 'gamma': [0.9,1,1.1], 'eta': [0.01, 0.05]}

gsdmm_param={'K': range(7,25), 'alpha': [0.1, 0.25], 'beta': [0.1, 0.25]}

print(len(combine(lda_param)))
print(len(combine(hdp_param)))
print(len(combine(gsdmm_param)))

135
486
72


In [9]:
for params in param_combinations:
    param_set = dict(zip(param_names, params))
    print(param_set)



{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 0.9, 'gamma': 0.9, 'eta': 0.01}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 0.9, 'gamma': 0.9, 'eta': 0.05}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 0.9, 'gamma': 1, 'eta': 0.01}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 0.9, 'gamma': 1, 'eta': 0.05}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 0.9, 'gamma': 1.1, 'eta': 0.01}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 0.9, 'gamma': 1.1, 'eta': 0.05}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 1, 'gamma': 0.9, 'eta': 0.01}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 1, 'gamma': 0.9, 'eta': 0.05}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 1, 'gamma': 1, 'eta': 0.01}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 1, 'gamma': 1, 'eta': 0.05}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 1, 'gamma': 1.1, 'eta': 0.01}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 1, 'gamma': 1.1, 'eta': 0.05}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 1.1, 'gamma': 0.9, 'eta': 0.01}
{'kappa': 1, 'K': 10, 'T': 100, 'alpha': 1.1, 'gamma': 0.9, 'eta': 0.05

In [ ]:
    def coherence(self, tokenized_dataset, metrics=['c_v', 'c_npmi', 'c_uci', 'u_mass', 'c_we'], glove_path='glove/glove.6B/glove.6B.300d.txt'):
        """ Get the coherence of the topic mode.

        :param tokenized_dataset: List of of documents, documents are lists of doc elements
        :param metric: Metric for computing the coherence, among <c_v, c_npmi, c_uci, u_mass, c_we>
         """
        results = defaultdict(dict)
        for metric in metrics:
            if metric not in ['c_v', 'c_npmi', 'c_uci', 'u_mass', 'c_we']:
                raise RuntimeError('Unrecognised metric: ' + metric)

            topic_words = [x['words'] for x in self.topics()]

            self.log.debug('loading dataset')

            dictionary = gensim.corpora.hashdictionary.HashDictionary(tokenized_dataset)


            if metric == 'c_we':

                if os.path.exists(glove_path): #.replace('txt', 'pickle')
                    glove = pickle.load(open(glove_path.replace('txt', 'pickle'), 'rb'))
                else:
                    w2v = utils.get_tmpfile("w2v")
                    glove2word2vec(glove_path, w2v)
                    glove = KeyedVectors.load_word2vec_format(w2v)
                    pickle.dump(glove, open(glove_path, 'wb'))

                results[metric]['c_we_per_topic'] = []

                for topic in topic_words:
                    score = 0
                    count = 0
                    for word1 in topic:
                        for word2 in topic:
                            if word1 == word2: continue
                            if word1 not in glove or word2 not in glove: continue
                            score += glove.similarity(word1, word2)
                            count += 1

                    results[metric]['c_we_per_topic'].append(0 if count == 0 else score / count)
                results[metric]['c_we'] = np.mean(results[metric]['c_we_per_topic'])
                results[metric]['c_we_std'] = np.std(results[metric]['c_we_per_topic'])

            while True:
                try:
                    self.log.debug('creating coherence model')
                    coherence_model = gensim.models.coherencemodel.CoherenceModel(topics=topic_words, texts=tokenized_dataset,
                                                                                dictionary=dictionary,
                                                                                coherence=metric)
                    coherence_per_topic = coherence_model.get_coherence_per_topic()

                    topic_coherence = [coherence_per_topic[i] for i, t in enumerate(self.topics())]

                    results[metric][metric + '_per_topic'] = topic_coherence
                    results[metric][metric] = np.nanmean(coherence_per_topic)
                    results[metric][metric + '_std'] = np.nanstd(coherence_per_topic)

                    break

                except KeyError as e:
                    key = str(e)[1:-1]
                    for x in topic_words:
                        if key in x:
                            x.remove(key)

        return results

In [ ]:
                except KeyError as e:
                    keyerrs+=1
                    self.log.info(f'key error ocurred. nb{keyerrs}')
                    key = str(e)[1:-1]
                    for x in topic_words:
                        if key in x:
                            x.remove(key)